In [1]:
import numpy as np
import pandas as pd

In [5]:
data = pd.DataFrame()

for yr in range(2022,2024):
    for mth in range(1,13):
        fname = f"vix_eod_{yr}{str(mth).zfill(2)}.txt"
        t = pd.read_csv(f"/kaggle/input/vix-options-data/{fname}")
        data = pd.concat([data, t])

In [9]:
data.columns

Index(['[QUOTE_UNIXTIME]', ' [QUOTE_READTIME]', ' [QUOTE_DATE]',
       ' [QUOTE_TIME_HOURS]', ' [UNDERLYING_LAST]', ' [EXPIRE_DATE]',
       ' [EXPIRE_UNIX]', ' [DTE]', ' [C_DELTA]', ' [C_GAMMA]', ' [C_VEGA]',
       ' [C_THETA]', ' [C_RHO]', ' [C_IV]', ' [C_VOLUME]', ' [C_LAST]',
       ' [C_SIZE]', ' [C_BID]', ' [C_ASK]', ' [STRIKE]', ' [P_BID]',
       ' [P_ASK]', ' [P_SIZE]', ' [P_LAST]', ' [P_DELTA]', ' [P_GAMMA]',
       ' [P_VEGA]', ' [P_THETA]', ' [P_RHO]', ' [P_IV]', ' [P_VOLUME]',
       ' [STRIKE_DISTANCE]', ' [STRIKE_DISTANCE_PCT]'],
      dtype='object')

In [10]:
cols = ['Time_Unix','Datetime','Date','Time','VIX_Close','Expiry_Date','Expiry_Unix','Dte','C_Delta','C_Gamma','C_Vega','C_Theta','C_Rho','C_IV','C_Volume','C_Last','C_Size','C_Bid','C_Ask','Strike','P_Bid','P_Ask','P_Size','P_Last','P_Delta','P_Gamma','P_Vega','P_Theta','P_Rho','P_IV','P_Volume','Strike_Distance','Strike_Dist_Percentage']
dropcols = ['Time_Unix','Datetime','Time','Expiry_Unix','C_Gamma','C_Vega','C_Theta','C_Rho','C_Size','P_Size','P_Gamma','P_Vega','P_Theta','P_Rho','P_Volume','Strike_Dist_Percentage']

In [11]:
data.columns = cols

In [13]:
data = data.drop(columns=dropcols)

In [20]:
data = data.reset_index().drop(columns='index')

In [38]:
data = data.drop(columns='C_Volume')

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324563 entries, 0 to 324562
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date             324563 non-null  object 
 1   VIX_Close        324563 non-null  float64
 2   Expiry_Date      324563 non-null  object 
 3   Dte              324563 non-null  float64
 4   C_Delta          324563 non-null  object 
 5   C_IV             324563 non-null  object 
 6   C_Last           324563 non-null  object 
 7   C_Bid            324563 non-null  object 
 8   C_Ask            324563 non-null  object 
 9   Strike           324563 non-null  float64
 10  P_Bid            324563 non-null  object 
 11  P_Ask            324563 non-null  object 
 12  P_Last           324563 non-null  object 
 13  P_Delta          324563 non-null  object 
 14  P_IV             324563 non-null  object 
 15  Strike_Distance  324563 non-null  float64
dtypes: float64(4), object(12)
memory usage

In [42]:
ccols = ['C_Delta','C_IV','C_Last','C_Bid','C_Ask','P_Bid','P_Ask','P_Last','P_Delta','P_IV']

In [43]:
for _col in ccols:
    data[_col] = pd.to_numeric(data[_col], errors='coerce')

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324563 entries, 0 to 324562
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date             324563 non-null  object 
 1   VIX_Close        324563 non-null  float64
 2   Expiry_Date      324563 non-null  object 
 3   Dte              324563 non-null  float64
 4   C_Delta          324558 non-null  float64
 5   C_IV             279047 non-null  float64
 6   C_Last           324468 non-null  float64
 7   C_Bid            324468 non-null  float64
 8   C_Ask            324468 non-null  float64
 9   Strike           324563 non-null  float64
 10  P_Bid            324468 non-null  float64
 11  P_Ask            324468 non-null  float64
 12  P_Last           324468 non-null  float64
 13  P_Delta          324556 non-null  float64
 14  P_IV             238684 non-null  float64
 15  Strike_Distance  324563 non-null  float64
dtypes: float64(14), object(2)
memory usage

In [47]:
data = data.fillna(0)

In [52]:
for _col in ['Date','Expiry_Date']:
    data[_col] = data[_col].str.strip()

In [58]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data['Expiry_Date'] = pd.to_datetime(data['Expiry_Date'], format='%Y-%m-%d')

In [67]:
data.to_csv('optionChainData.csv', index=False)

In [68]:
netVal = 1000000
cash_percentage = 0.05

In [125]:
ID = 0

def idGenerate():
    ID += 1
    return ID

In [126]:
def resetIDs():
    ID = 0
    return None

In [69]:
def getMonthEnd(x: pd.Timestamp):
    return x + pd.offsets.BMonthEnd()


def getMonthBegin(x: pd.Timestamp):
    return x - pd.offsets.BMonthBegin()


def getNextMonthBegin(x: pd.Timestamp):
    return x + pd.offsets.BMonthBegin()

In [121]:
def get3MonthEnd(x: pd.Timestamp):
    t = x + pd.offsets.MonthEnd(3)
    if t.weekday() == 6:
        return t - pd.Timedelta(2, unit='D')
    elif t.weekday() == 5:
        return t - pd.Timedelta(1, unit='D')
    else:
        return t
    return None


def get4MonthBegin(x: pd.Timestamp):
    t = x + pd.offsets.MonthBegin(3)
    if t.weekday() == 6:
        return t + pd.Timedelta(1, unit='D')
    elif t.weekday() == 5:
        return t + pd.Timedelta(2, unit='D')
    else:
        return t
    return None

In [73]:
dateList = sorted(data['Date'].unique())
expiryDateList = sorted(data['Expiry_Date'].unique())

In [81]:
def getOrderBook():
    for dt in dateList:
        tdf = data[data['Date'] == dt]
        tdf = tdf.reset_index().drop(columns='index')
        yield tdf

In [133]:
class Queue:
    def __init__(self):
        self.queue = []
        return None
    
    def enqueue(self, item):
        self.queue.append(item)
        return None
    
    def dequeue(self):
        if len(self.queue) > 0:
            t = self.queue.pop(0)
            return t
        else:
            raise Exception("Queue is Empty!")
            return None
        return None
    
    def dequeueById(self, id):
        _arg = None
        for i in range(len(self.queue)):
            if id == self.queue[i].id:
                _arg = self.queue.index(i)
        if _arg is not None:
            t = self.queue.pop(_arg)
            return t
        else:
            raise Exception("ID Doesnt exist in Queue!")
            return None
        return None
    
    def peak(self, ln=1):
        if len(self.queue) > 0:
            return self.queue[:ln]
        else:
            raise Exception("Queue is Empty!")
            return None
        return None
    
    def items(self):
        if len(self.queue) > 0:
            for i in range(len(self.queue)):
                yield self.queue[i]
        else:
            raise Exception("Queue is Empty!")
        return None
    
    def sortedQueue(self):
        return sorted(self.queue, key = lambda x: x.entry_time)

In [135]:
class Straddle:
    def __init__(self, _id, entry_time, expiry_time, strike, c_bid, c_ask, p_bid, p_ask, lots, option_size=100):
        self.id = _id
        self.entry_time = entry_time
        self.expiry_time = expiry_time
        self.strike = strike
        self.c_bid = c_bid
        self.c_ask = c_ask
        self.p_bid = p_bid
        self.p_ask = p_ask
        self.lots = lots
        self.trade_value = (c_bid + p_bid) * self.lots * option_size
        return None

In [ ]:
class VRP:
    def __init__(self):
        try:
            self.invested = False
            self.portfolio = netVal
            self.cash_percentage = cash_percentage
            self.order_book = None
            self.available_balance = 0
            self.option_size = 100
            self. unused_balance = 0
            self.current_lots = 0
            self.strike = 0
            self.call_range = 0
            self.put_range = 0
            self.entry_time = None
            self.c_bid = 0
            self.p_bid = 0
            self.c_ask = 0
            self.p_ask = 0
            self.call_delta = None
            self.put_delta = None
            self.pf_delta = None
            self.ids = []
            self.q = Queue()
            self.end_of_3month = pd.Timestamp(year=1990, month=1, day=1)
            self.next_4month_begin = pd.Timestamp(year=1990, month=1, day=1)
            self.expiry_date = pd.Timestamp(year=1990, month=1, day=1)
            self.expiry_date_list = None
            self.past_expiry_list = []
            self.exit_time = None
            self.entry_trades = pd.DataFrame()
            self.exit_trades = pd.DataFrame()
            self.reldf = None
        except Exception as e:
            print(f"ERROR IN INITIALIZING VRP CLASS: \n\t{str(e)}\n")
            exit()
        return None
    
    
    def get_3MonthEnd(self):
        self.end_of_3month = get3MonthEnd(self.entry_time)
        return None
    
    
    def get_4MonthBegin(self):
        self.next_4month_begin = get4MonthBegin(self.entry_time)
        return None
    
    
    def get3MonthExpiry(self):
        for i in range(1, len(self.expiry_date_list)):
            if self.end_of_3month == self.expiry_date_list[i]:
                return self.expiry_date_list[i]
            elif self.end_of_3month < self.expiry_date_list[i]:
                return self.expiry_date_list[i-1]
            else:
                pass
        return None
        
    
    def main(self):
        for idx, order_book in enumerate(getOrderbook()):
            self.entry_time = pd.Timestamp(order_book['Date'].unique()[0])
            self.end_of_3month = self.get_3MonthEnd()
            self.next_4month_begin = self.get_4MonthBegin()
            self.expiry_date_list = sorted(order_book['Expiry_Date'].unique())
            self.expiry_date = self.get3MonthExpiry()
            self.reldf = order_book[order_book['Expiry_Date'] == self.expiry_date]
            
            min_strike_dist = self.reldf['Strike_Dist'].argmin()
            
            
            if self.expiry_date in self.past_expiry_list:
                # TRAVERSE QUEUE, MARK TO MARKET FOR EACH, SQUARE OFF IF EXPIRY REACHED, IF INVESTED
                if self.invested:
                    for conts in q.items():
                        s_expiry_time = conts.expiry_time
                        if s_expiry_time < self.entry_time:
                            # TODO: ADD LOGIC TO DEQUEU USING ID
                            t = q.dequeueById(conts.id)
                            continue
                        else:
                            if s_expiry_time == self.entry_time:
                                # SQUARE POSITION OFF, MARK TO MARKET, HEDGE, DEQUEUE CONTRACT
                                pass
                            else:
                                # MARK TO MARKET, HEDGE
                                pass
                else:
                    # MARK TO MARKET OTHER TRADES OR SIT IDLE
                    pass
            else:
                self.past_epxiry_list.append(self.expiry_date)
                # LOGIC TO SELL STRADDLE AND HEDGE FOR FIRST TIME EXPIRY DATE
                strike = self.reldf.at[min_strike_dist, 'Strike']
                c_bid = self.reldf.at[min_strike_dist, 'C_Bid']
                c_ask = self.reldf.at[min_strike_dist, 'C_Ask']
                p_bid = self.reldf.at[min_strike_dist, 'P_Bid']
                p_ask = self.reldf.at[min_strike_dist, 'P_Ask']
                c_delta = self.reldf.at[min_strike_dist, 'C_Delta']
                p_delta = self.reldf.at[min_strike_dist, 'P_Delta']
                self.available_balance = int((self.portfolio * (1 - self.cash_percentage))/3) # 3 for 3 months -> make variable?
                self.current_lots = int((self.available_balance / (c_bid + p_bid)) / self.option_size)               
                _id = idGenerate()
                new_contract = Straddle(_id, self.entry_time, self.expiry_date, strike, c_bid, c_ask, p_bid, p_ask, self.current_lots, self.option_size)
                self.q.enqueue(new_contract)
                # TRAVERSE QUEUE, SELL STRADDLE, MARK TO MARKET, CALCULATE DELTA AND HEDGE